In [17]:
# загрузим библиотеки и установим опции

from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

In [1]:
!ls

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [3]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

In [4]:
# загрузим словарик сайтов
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'всего сайтов:', sites_dict.shape[0])
sites_dict.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [5]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


In [6]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [7]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [8]:
def calc_tf(row):
    vc = row.value_counts()
    vc = vc[vc.index != 0]
    tf = vc / vc.sum()
    return tf

In [9]:
def calc_idf(df):
    max_index = df.max().max()
    term_counts = np.zeros(max_index + 1)
    def update_counts(row):
        for i in set(row):
            term_counts[i] += 1
    df.apply(update_counts, axis = 1)
    return np.log(df.shape[0] / term_counts)

In [10]:
def calc_tfidf(df):
    idf = calc_idf(df)
    def calc_tf(row):
        vc = row.value_counts()
        vc = vc[vc.index != 0]
        tf = vc / vc.sum()
        return tf
    indptr = [0]
    indicies = []
    data = []
    def process_row(row):
        tf = calc_tf(row)
        for i in tf.index:
            indicies.append(i)
            data.append(tf[i] * idf[i])
        indptr.append(len(indicies))
    df.apply(process_row, axis = 1)
    return csr_matrix((data, indicies, indptr))

In [11]:
def calc_counts(df):
    indptr = [0]
    indicies = []
    data = []
    def process_row(row):
        vc = row.value_counts()
        for i in vc.index:
            indicies.append(i)
            data.append(vc[i])
        indptr.append(len(indicies))
    df.apply(process_row, axis = 1)
    return csr_matrix((data, indicies, indptr))

In [12]:
%%time 
count_matrix = calc_counts(full_sites)

Wall time: 2min 34s


In [13]:
tfidftrans = TfidfTransformer()
tfidf = tfidftrans.fit_transform(count_matrix)

In [14]:
def get_auc_lr_valid(X, y, C=1, seed=17, ratio = 0.9):
    # разделим выборку на обучающую и валидационную
    idx = round(X.shape[0] * ratio)
    # обучение классификатора
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    # прогноз для валидационной выборки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # считаем качество
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [15]:
%%time
# выделим из объединенной выборки только обучающую (для которой есть ответы)
X_train = tfidf[:idx_split, :]

# считаем метрику на валидационной выборке
print(get_auc_lr_valid(X_train, y_train))

0.927145981576
Wall time: 1.06 s


In [25]:
weekday = DictVectorizer().fit_transform([{time.weekday():True} for time in full_df['time1']])

hour = DictVectorizer().fit_transform([{time.hour:True} for time in full_df['time1']])

# mounth = DictVectorizer().fit_transform([{time.month + time.year :True} for time in full_df['time1']])

In [19]:
# датафрейм для новых признаков
full_new_feat = pd.DataFrame(index=full_df.index)

# добавим признак start_month
full_new_feat['start_month'] = full_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
full_new_feat['start_hour'] = full_df['time1'].apply(lambda ts: ts.hour)
full_new_feat['morning'] = full_new_feat['start_hour'].apply(lambda hour: int(hour <= 11))
tmp_scaled = StandardScaler().fit_transform(full_new_feat[['start_month', 'start_hour', 'morning']])

In [33]:
X_train = csr_matrix(hstack([tfidf[:idx_split,:], weekday[:idx_split,:], hour[:idx_split,:]]))
X_test = csr_matrix(hstack([tfidf[idx_split:,:], weekday[idx_split:,:], hour[idx_split:,:]]))

In [34]:
%%time
# набор возможных значений С
Cs = np.logspace(-3, 1, 10)

scores = []
    
for C in Cs:
    scores.append(get_auc_lr_valid(X_train, y_train, C=C))

Wall time: 11.1 s


In [35]:
C = Cs[scores.index(max(scores))]
scores, C

([0.95568874722838126,
  0.96334239996811744,
  0.96734373565883613,
  0.97277496087126647,
  0.9762965468892657,
  0.97885440669246271,
  0.98109494937418174,
  0.98252846192677612,
  0.98311056222193238,
  0.98304172463516082],
 3.5938136638046259)

In [36]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [37]:
# обучим модель на всей выборке с оптимальным коэффициентом регуляризации
lr = LogisticRegression(C=C, random_state=17).fit(X_train, y_train)

# сделаем прогноз для тестовой выборки
y_test = lr.predict_proba(X_test)[:, 1]

# запишем его в файл, готовый для сабмита
write_to_submission_file(y_test, 'baseline.csv')

In [ ]:
%%time
# набор возможных значений С
Cs = np.logspace(-3, 1, 10)

scores = []
    
for C in Cs:
    scores.append(get_auc_lr_valid(X_train, y_train, C=C))

In [ ]:
list(zip(scores, Cs))

In [24]:
mounth

<336358x13 sparse matrix of type '<class 'numpy.float64'>'
	with 336358 stored elements in Compressed Sparse Row format>

In [31]:
train_df['time1'].min(), train_df['time1'].max()

(Timestamp('2013-01-12 08:05:57'), Timestamp('2014-04-30 23:39:53'))

In [32]:
test_df['time1'].min(), test_df['time1'].max()

(Timestamp('2014-05-01 17:14:03'), Timestamp('2014-12-05 23:26:53'))